In [1]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import numpy.ma as ma
import matplotlib.mlab as mlab
from scipy.stats import norm
from scipy.optimize import curve_fit
from collections import defaultdict
from datetime import datetime
import os
import glob
import csv
import numpy as h5py 


In [2]:
%pylab
%matplotlib qt

Using matplotlib backend: Qt4Agg
Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [6]:
os.chdir('../telescope_control/data_aquisition/weather_data/TSI_data/uncompressed')

In [ ]:
YEAR = [2012,2013,2014]
numfiles = 0
for y in YEAR:
    fpy = 0
    files = glob.glob('*%s*' % y)

    for f in files:
        #y = int(f[3:7])
        m = f[7:9]
        d = f[9:]

        date = datetime.datetime(y, int(m), int(d))
        
        if date < datetime.datetime(y, 6, 1) or date > datetime.datetime(y, 8, 31):
            continue
            
        os.chdir('tsi%d%s%s' % (y, m, d))
        files = glob.glob('*')
        
        for fi in files:
            fpy += 1
            numfiles += 1
        os.chdir('../')     
    
    print ' '
    print 'files in year %d: %d' % (y, fpy) 
    print 'done with %d' % y, ', files so far: ', numfiles
    print ' '

In [7]:
print(os.getcwd())
#os.chdir('../')
#print os.getcwd()

/media/arikaplan/greenpol_2/software_git_repos/greenpol/telescope_control/data_aquisition/weather_data/TSI_data/uncompressed


In [9]:
clearsky = defaultdict(dict)
dates = defaultdict(dict)
YEAR = [2012,2013,2014]
#os.chdir('../telescope_control/data_aquisition/weather_data/TSI_data/uncompressed')
count = 0
for y in YEAR:
    time = []
    csky = []
    files = glob.glob('*%s*' % y)

    for f in files:
        #y = int(f[3:7])
        m = f[7:9]
        d = f[9:]
        
        date = datetime.datetime(y, int(m), int(d))
        
        if date < datetime.datetime(y, 6, 1) or date > datetime.datetime(y, 8, 31):
            continue
            
        os.chdir('tsi%d%s%s' % (y, m, d))
        files = glob.glob('*.properties')
        
        for fi in files:
            ff = open(fi, 'r')
            lines = ff.readlines()
            try:
                opaque = float(lines[9][26:])
                thin = float(lines[5][24:])
                sky = 1. - opaque - thin
            except:
                sky = -1
                
            csky.append(sky)
            time.append(lines[0])
            ff.close()
            
            count += 1
            sys.stdout.write("\rPercent complete: %.2f" % (float(count) * 100. / float(numfiles)))
            sys.stdout.flush()

        os.chdir('../')     
    
    print ' '
    print 'done with %d' % y  
    print ' '
    clearsky[y] = csky
    dates[y] = time

Percent complete: 33.23 
done with 2012
 
Percent complete: 66.58 
done with 2013
 
Percent complete: 100.00 
done with 2014
 


In [10]:
#write data to txt file
for y in [2012,2013,2014]:
    lists = zip(['Dates'], ['Clear Sky Fraction']) + zip(dates[y], clearsky[y])
    print y
    file = open('../sky_fraction/%d.txt' % y, 'w')
    for i in range(len(lists)):
        file.write(str(lists[i])[1:-1]+'\n')
    file.close

2012
2013
2014


In [8]:
#read data from txt file
times = defaultdict(dict)
clearsky = defaultdict(dict)
YEAR = [2012, 2013, 2014, 2015, 2016, 2017, 2018]
for y in YEAR:
    print(y)
    time = []
    csky = []
    with open('../sky_fraction/%d.txt' % y, 'r') as inputfile:
        next(inputfile)
        for line in inputfile:
            l = line.split(',')
            try:
                time.append(l[0])
                csky.append(float(l[1]))
            except:
                pass
    times[y] = time
    clearsky[y] = csky

2012
2013
2014
2015
2016
2017
2018


In [9]:
def nummonth(name):
    if name == 'Jun':
        num = 6
    elif name == 'Jul':
        num = 7
    elif name == 'Aug':
        num = 8
    else:
        print(name)
    return num

In [10]:
dates = defaultdict(dict)

for y in YEAR:
    d = []
    for t in range(len(times[y])):
        month = nummonth(times[y][t][6:9])
        day = times[y][t][10:12]
        hour = times[y][t][13:15]
        minute = times[y][t][16:18]
        second = times[y][t][19:21]
    
        d.append(datetime.datetime(y, month, int(day), int(hour), int(minute), int(second)))
        
    dates[y] = np.asarray(d)

In [97]:
y = 2018
threshhold = 0.8

cskyperiod = defaultdict(dict)
cloudperiod = defaultdict(dict)
csky = defaultdict(dict)
csp = []
cp = []
cs = []
j = 0


for i in range(len(clearsky[y])-1):
    #sys.stdout.write("\rPercent complete: %.2f" % (float(i) * 100. / float(len(clearsky[y]))))
    #sys.stdout.flush()
    if (clearsky[y][i] >= threshhold) and (clearsky[y][i+1] >= threshhold):
        continue
    else:
        duration = (dates[y][i] - dates[y][j]).total_seconds()
        if duration == 0.0:
            duration = 60.
        csp.append(duration)
        j=i+1
    '''
    if clearsky[y][i] < threshhold and clearsky[y][i+1] < threshhold:
        continue
    else:
        duration = (dates[y][i] - dates[y][j]).total_seconds()
        if duration == 0.0:
            duration = 60.
        if duration < 0.:
            print(duration, dates[y][i], dates[y][j], i)
            #break
        cp.append((dates[y][i] - dates[y][j]).total_seconds())
        j=i+1
   '''    
print(len(csp))
#print(len(cp))
cskyperiod[y] = np.asarray(csp)
#cloudperiod[y] = np.asarray(cp)
t = cskyperiod[2018]/3600.

79359


In [99]:
figure()
#t = cskyperiod[2018]/3600.
plot(t, 'b-')
ylabel('hours')

In [100]:
print(len(np.where(t<=1.)[0]))

79207


In [69]:
print(cskyperiod[y].sum()/3600./24.)
print(28/90)
print(cskyperiod[y].max())

83.483125
0.3111111111111111
129360.0


In [106]:
print(type(a), type(t))

<class 'numpy.ndarray'> <class 'numpy.ndarray'>


In [117]:
figure()
bins = 100
plt.hist(t, bins = bins)
ylim(0,a.max())
show()

In [114]:
figure()
plot(values)

In [115]:
#create cumulative distribution:
figure()
bins=100
#a = np.hstack(t)
values, base = np.histogram(t, bins = bins, density=True)
cumulative = np.cumsum(values)
cumu = cumulative/cumulative.max()
#print(cumulative.max())
cumu_I = (1. - cumu)

plot(base[:-1], (cumu_I)*100, 'k-', linewidth = 2, label ='%d' % y)
    

plt.title('June - August: Prob(clear sky fraction $>$ 80% for $>$ x hours)')
plt.xlabel('hours')
plt.ylabel('Cumulative Distribution (%)')
#plt.legend([c_15, c_14, c_13, c_12, c_11, c_10, c_09], ['2015', '2014', '2013', '2012', '2011', '2010', '2009'], loc = 'lower right')
legend()
#plt.xlim([0,cs.max()])
#plt.ylim([0, 105])
#plt.savefig('../meteorology/plots/year_to_year_comparisons/wind_speed_hourly_average_1-cumulative_dist.png')
plt.show()

ValueError: x and y must have same first dimension

In [87]:
for y in YEAR:
    clearsky[y] = ma.masked_equal(clearsky[y], -1)

In [88]:
cs_good = defaultdict(dict)
#create arrays with missing elements deleted
for y in YEAR:
    
    s = clearsky[y]
    igood = np.where(s >= 0.)[0]
    cs_good[y] = s[igood]

In [89]:
figure()
plot(cs_good[y])

In [90]:
plt.hist(cs_good[y], bins = 50)

(array([ 22794.,  15709.,   4581.,   2297.,   1689.,   1313.,   1002.,
           878.,    888.,    787.,    774.,    677.,    668.,    658.,
           639.,    655.,    627.,    565.,    609.,    546.,    562.,
           559.,    576.,    548.,    568.,    578.,    624.,    604.,
           568.,    646.,    670.,    672.,    652.,    736.,    726.,
           863.,    866.,    992.,   1149.,   1161.,   1304.,   1713.,
          1875.,   2310.,   2149.,   2158.,   2868.,   3699.,   5301.,
         16379.]),
 array([ 0.  ,  0.02,  0.04,  0.06,  0.08,  0.1 ,  0.12,  0.14,  0.16,
         0.18,  0.2 ,  0.22,  0.24,  0.26,  0.28,  0.3 ,  0.32,  0.34,
         0.36,  0.38,  0.4 ,  0.42,  0.44,  0.46,  0.48,  0.5 ,  0.52,
         0.54,  0.56,  0.58,  0.6 ,  0.62,  0.64,  0.66,  0.68,  0.7 ,
         0.72,  0.74,  0.76,  0.78,  0.8 ,  0.82,  0.84,  0.86,  0.88,
         0.9 ,  0.92,  0.94,  0.96,  0.98,  1.  ]),
 <a list of 50 Patch objects>)

In [79]:
figure()

plot(clearsky[2017], linewidth = 2, label='2017')
plot(clearsky[2018], linewidth = 2, label='2018')
legend()

In [60]:
figure()
plot(cs_good[y])

In [91]:
#create cumulative distribution:

base = defaultdict(dict)
cumu = defaultdict(dict)
cumu_I = defaultdict(dict)

bins = 100

figure()
for y in [2018]:
    
    cs = cs_good[y]
    values, base[y] = np.histogram(cs, bins = bins, density=True)
    cumulative = np.cumsum(values)
    cumu[y] = cumulative/cumulative.max()
    cumu_I[y] = (1. - cumu[y])
    
    if y == 2018:
        plot(base[y][:-1], (cumu_I[y])*100., 'k-', linewidth = 2, label ='%d' % y)
    else:    
        plot(base[y][:-1], (cumu_I[y])*100., '--', linewidth = 2, label ='%d' % y)
    

plt.title('June - August: Prob(clear sky fraction $>$ x)')
plt.xlabel('clear sky percentage')
plt.ylabel('Cumulative Distribution (%)')
#plt.legend([c_15, c_14, c_13, c_12, c_11, c_10, c_09], ['2015', '2014', '2013', '2012', '2011', '2010', '2009'], loc = 'lower right')
legend()
#plt.xlim([0,cs.max()])
plt.ylim([0, 105])
#plt.savefig('../meteorology/plots/year_to_year_comparisons/wind_speed_hourly_average_1-cumulative_dist.png')
plt.show()

In [4]:
from PIL import Image

In [8]:
im = Image.open('G:/software_git_repos/greenpol/telescope_control/data_aquisition/weather_data/TSI_data/uncompressed/tsi20160705/20160705022700.jpg')
#im = Image.open('G:/software_git_repos/greenpol/telescope_control/data_aquisition/weather_data/TSI_data/uncompressed/tsi20160705/20160705002100.jpg')

pix = im.load()
x, y = im.size
print x,y  # Get the width and hight of the image for iterating over
print pix[x-1,y-1]  # Get the RGBA Value of the a pixel of an image
#pix[x,y] = value  # Set the RGBA Value of the image (tuple)

288 352
(108, 94, 153)


In [9]:
xx,yy  = np.arange(0, x), np.arange(0., y)
X,Y = np.meshgrid(xx,yy)

z = np.zeros(x*y)
sig = np.reshape(z, (y, x))
count = 0.
for iy in range(y):
    for ix in range(x):
        sig[iy][ix] = pix[ix,iy][0]*pix[ix,iy][1]*pix[ix,iy][2]/1e7
        if sig[iy][ix] <= 1:
            count +=1.

print count/x/y

0.990806502525


In [10]:
fig, ax = subplots()
pcolormesh(X, Y, sig)

colorbar(label = 'R*G*B value')
clim(sig.min(),sig.max())
axis([X.min(), X.max(), Y.max(), Y.min()])
ylabel('Y')
xlabel('X')
title('cloudy day image')
grid()


In [3]:
print os.getcwd()

G:\software_git_repos\greenpol\notebook


In [8]:
folder = '../telescope_control/data_aquisition/weather_data/PWV/'
pwv = h5py.File(folder + 'asdfint1p5_pwv.mat', 'r')
doy = h5py.File(folder + 'asdfint1p5_doy.mat', 'r')
cloud = h5py.File(folder + 'asdfint1p5_cloud.mat', 'r')

AttributeError: 'module' object has no attribute 'File'

In [ ]:
f = h5py.File('somefile.mat','r') 
data = f.get('data/variable1') # Get a certain dataset
data = np.array(data)